<a href="https://colab.research.google.com/github/federicOO1/LAB-IA/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import librerie


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import matplotlib.pyplot as plt
from torchvision import transforms
import os
import numpy as np
import PIL

In [2]:
!pip install rasterio
import rasterio
from rasterio.plot import reshape_as_image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 36.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.chdir("/content/drive/MyDrive/PotsdamDataset")

In [5]:
class PotsdamDataset(Dataset):
    def __init__(self, dataset_folder):
        self.dataset_folder = dataset_folder
        self.image_paths = []
        self.world_file_paths = []
        self.mask_paths = []

        # Leggi i percorsi delle immagini, dei file .tfw e delle maschere
        end_RGBIR_folder = os.listdir(dataset_folder)[0]
        end_LABELS_folder = os.listdir(dataset_folder)[1]

        RGBIR_folder = data_folder + '/' + end_RGBIR_folder
        LABELS_folder = data_folder + '/' + end_LABELS_folder

        for file_name in os.listdir(RGBIR_folder):
            if file_name.endswith('.tif'):
                image_path = os.path.join(RGBIR_folder, file_name)
                world_file_path = os.path.join(RGBIR_folder, file_name.replace('.tif', '.tfw'))

                if os.path.exists(world_file_path):
                    self.image_paths.append(image_path)
                    self.world_file_paths.append(world_file_path)


        for label_name in os.listdir(LABELS_folder):
            mask_path = os.path.join(LABELS_folder, label_name)

            if os.path.exists(mask_path):
              self.mask_paths.append(mask_path)

    def __len__(self):
        return len(self.image_paths)

    def load_world_file(self, world_file_path):
          lines = open(world_file_path).readlines()
          try:
              parameters = [float(line.strip()) for line in lines if line.strip()]
              if len(parameters) == 6:
                  return parameters
              else:
                  raise ValueError("Il file .tfw non contiene 6 parametri.")
          except Exception as e:
              print(f"Errore durante la lettura dei parametri di georeferenziazione: {str(e)}")
              return None

    def convert_labels_to_tensor(self, rgb_label):
      color_image = reshape_as_image(rgb_label)
      # Definisci i colori delle classi nella maschera RGB
      colors_to_labels = {
          (255, 255, 255): 0,  # Impervious surfaces -> Classe 0
          (0, 0, 255): 1,  # Building -> Classe 1
          (0, 255, 255): 2,  # Low vegetation -> Classe 2
          (0, 255, 0): 3,  # Tree -> Classe 3
          (255, 255, 0): 4,  # Car -> Classe 4
          (255, 0, 0): 5  # Clutter/background -> Classe 5
      }

      # Inizializza un'immagine di etichette con valori di sfondo (0)
      height, width, _ = color_image.shape
      class_label = np.zeros((height, width), dtype=np.int32)

      for i in range(height):
          for j in range(width):
              pixel_color = tuple(color_image[i, j])
              #print(pixel_color)
              class_label[i, j] = colors_to_labels.get(pixel_color, -1)  # -1 per i colori non mappati
              #print(class_label[i, j])


      # Stampa l'array di classi risultante
      class_label_tensor = torch.tensor(class_label, dtype=torch.long)

      return class_label_tensor

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        world_file_path = self.world_file_paths[idx]
        mask_path = self.mask_paths[idx]

        # Carica l'immagine TIFF utilizzando la libreria rasterio
        image = rasterio.open(image_path).read()

        # Carica i parametri di georeferenziazione dal file .tfw
        world_params = self.load_world_file(world_file_path)

        # Carica la maschera con rasterio
        mask = rasterio.open(mask_path).read()

        # Converti la maschera RGB nel formato appropriato per CrossEntropyLoss
        label = self.convert_labels_to_tensor(mask)

        # Restituisci l'immagine, la maschera e i parametri di georeferenziazione
        return image, label, world_params


In [6]:
# Definisci il percorso della cartella contenente i dati .tif e .tfw
data_folder = "/content/drive/MyDrive/PotsdamDataset"

# Crea un'istanza del dataset
dataset = PotsdamDataset(data_folder)